## **Simulando o Feixe**

Esse trabalho tem como objetivo simular o comportamento de um feixe de elétrons em acelerador linear, bem como a sua correção utilizando eletroímãs de quadrupolo.



In [1]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import gridspec
from matplotlib.animation import FuncAnimation
from matplotlib.widgets import Slider

# %matplotlib widget

plt.rcParams.update({
    "font.family": "serif",
    "mathtext.fontset": "cm",
    "font.size": 12 
})

Equações principais:

$$
\vec{F} = m \cdot \vec{a} = \frac{d\vec{p}}{dt}
$$

$$
\gamma = \sqrt{1 + \left(\frac{|\vec{v}|}{c} \right)^2} = \sqrt{1 + \left(\frac{|\vec{p}|}{m_0 c}\right)^2}
$$

$$
\vec{v} = \frac{\vec{v}}{\gamma} = \frac{\vec{p}}{m_0 \gamma} \approx c
$$

<!-- $$
\vec{v} = v_x \hat{i} + v_y \hat{j} + v_z \hat{k} \rightarrow |\vec{v}| = \sqrt{v_x^2 + v_y^2 + v_z^2} \approx c
$$ -->

$$
\vec{B} = B_x \hat{i} + B_y \hat{j} + B_z \hat{k}
$$

$$
\vec{F}^{N+1} = \vec{F}^{N} + \vec{f}_{ext} \cdot \Delta t = \vec{F}^{N} + q \cdot \vec{v} \times \vec{B} \cdot \Delta t
$$

In [6]:
config = {
    "V_LUZ": 2.99792458e8, # m\s
    "MASSA_E": 9.1093837e-31, # kg
    "CARGA_E": 1.60217663e-19, # C
    "HEIGHT": 100,
    "WIDTH": 100,
}

parametros = {
    "vx0": 0.0,
    "vy0": 0.0,
    "vz0": 1e-8 * config["V_LUZ"],
    "x0": config["WIDTH"] / 2,
    "y0": config["HEIGHT"] / 2,
    "z0": 0.0,
    "forca": [0, 0, 1e-16],
    "dt": 1e-11,
}       

In [ ]:
class FeixeEletron():
    def __init__(self, parametros, config):
        self.c = config["V_LUZ"]
        self.m_e = config["MASSA_E"]
        self.dt = parametros["dt"]

        self.forca = np.array(parametros["forca"], dtype=float)
        self.pos0 = np.array([parametros["x0"], parametros["y0"], parametros["z0"]], dtype=float)
        self.vel0 = np.array([parametros["vx0"], parametros["vy0"], parametros["vz0"]], dtype=float)

        v0_mag = np.linalg.norm(self.vel0)
        self.gamma0 = 1.0 / np.sqrt(1.0 - (v0_mag/self.c)**2)
        self.momento0 = self.m_e * self.vel0 * self.gamma0

    def cal_posicoes(self, n_times):
        n_passos = np.arange(1, n_times + 1) 
        self.variacao_tempo = n_passos * self.dt

        self.impulsos = np.outer(self.variacao_tempo, self.forca)

        self.momentos = self.impulsos + self.momento0
        momentos_norm = np.linalg.norm(self.momentos, axis=1, keepdims=True)

        self.gammas = np.sqrt(1.0 + (momentos_norm/(self.m_e * self.c))**2)

        velocidades = self.momentos / (self.gammas * self.m_e)
        self.vel = np.vstack([self.vel0, velocidades])

        self.deslocamentos = velocidades * self.dt
        
        posicoes = self.pos0 + np.cumulative_sum(self.deslocamentos, axis=0)
        self.pos = np.vstack([self.pos0, posicoes])

        self.t = np.arange(0, n_times + 1) * self.dt

    def runge_kutta():
        pass

N_VEZES = 50000

eletron = FeixeEletron(parametros, config)
eletron.cal_posicoes(N_VEZES)


In [ ]:
def runge_kutta():
    pos_k1 = 

In [ ]:
import numpy as np

class FeixeEletronOtimizado:
    def __init__(self, parametros, config):
        self.c = config["V_LUZ"]
        self.m_e = config["MASSA_E"]
        self.dt = parametros["dt"]
        
        # Garante que vetores sejam arrays numpy float
        self.forca = np.array(parametros["forca"], dtype=float)
        self.pos_0 = np.array([parametros["x0"], parametros["y0"], parametros["z0"]], dtype=float)
        self.vel_0 = np.array([parametros["vx0"], parametros["vy0"], parametros["vz0"]], dtype=float)
        
        # Cálculo inicial do momento (p0)
        v_mag = np.linalg.norm(self.vel_0)
        gamma_0 = 1.0 / np.sqrt(1.0 - (v_mag/self.c)**2)
        self.momento_0 = self.m_e * self.vel_0 * gamma_0

    def run(self, n_times):
        """
        Calcula toda a trajetória de uma vez usando vetorização.
        """
        # 1. Cria um array de passos de tempo: [1, 2, ..., N]
        steps = np.arange(1, n_times + 1)
        
        # 2. Calcula o impulso acumulado para cada passo (F * dt * passo)
        # O resultado é uma matriz (n_times, 3)
        # Se Força é (3,), impulsos vira (N, 3)
        impulsos = np.outer(steps, self.forca * self.dt)
        
        # 3. Calcula o Momento em todos os instantes (p = p0 + F*t)
        # Broadcasting: soma o vetor p0 a todas as linhas da matriz impulsos
        momentos = self.momento_0 + impulsos
        
        # 4. Calcula a Norma do Momento para todos os pontos
        # axis=1 calcula a norma ao longo das coordenadas x,y,z
        momentos_norm = np.linalg.norm(momentos, axis=1, keepdims=True)
        
        # 5. Calcula Gamma para todos os pontos (Vetorizado)
        gammas = np.sqrt(1.0 + (momentos_norm / (self.m_e * self.c))**2)
        
        # 6. Calcula Velocidade para todos os pontos
        # v = p / (m * gamma)
        velocidades = momentos / (gammas * self.m_e)
        
        # Adiciona a velocidade inicial no começo do array (opcional, para plotar t=0)
        velocidades = np.vstack([self.vel_0, velocidades])
        
        # 7. Calcula Posição acumulada (Integração)
        # r = r0 + soma_acumulada(v * dt)
        deslocamentos = velocidades[:-1] * self.dt # Remove o último v para alinhar dimensões
        trajetoria = self.pos_0 + np.cumsum(deslocamentos, axis=0)
        
        # Adiciona a posição inicial no topo
        self.trajetoria = np.vstack([self.pos_0, trajetoria])
        self.velocidades = velocidades
        
        # Cria vetor de tempo para plotagem
        self.t = np.arange(0, n_times + 1) * self.dt

# --- Exemplo de Uso ---
config = {
    "V_LUZ": 3e8,
    "MASSA_E": 9.11e-31
}

parametros = {
    "dt": 1e-11,
    "forca": [0.0, 1.0e-14, 0.0], # Força constante em Y
    "x0": 0, "y0": 0, "z0": 0,
    "vx0": 1e6, "vy0": 0, "vz0": 0
}

# Execução
eletron = FeixeEletronOtimizado(parametros, config)
eletron.run(100000)

print(f"Posição Final: {eletron.trajetoria[-1]}")

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation

# --- CONSTANTES FÍSICAS ---
c = 2.998e8       # Velocidade da luz (m/s)
m_e = 9.109e-31   # Massa de repouso (kg)
q_e = -1.602e-19  # Carga (C)

# --- CONFIGURAÇÃO DA SIMULAÇÃO ---
dt = 1e-11        # Passo de tempo (s) - muito curto pois v é alta
steps = 200       # Quantidade de passos

class ElectronBeam:
    def __init__(self, x0, y0, energia_v):
        # Estado inicial
        self.pos = np.array([x0, y0, 0.0], dtype=float) # Começa em z=0
        
        # Define momento inicial baseado na velocidade desejada (ex: 0.99c)
        v_z = energia_v * c
        gamma_inicial = 1 / np.sqrt(1 - (v_z/c)**2)
        
        # p = gamma * m * v
        self.momentum = np.array([0.0, 0.0, gamma_inicial * m_e * v_z])
        
        # Histórico para plotar o rastro
        self.traj_x = [x0]
        self.traj_y = [y0]
        self.traj_z = [0.0]

    def get_velocity(self):
        # Recupera velocidade real a partir do momento relativístico
        # Essa é a mágica para não ultrapassar a velocidade da luz
        p_mag = np.linalg.norm(self.momentum)
        gamma = np.sqrt(1 + (p_mag / (m_e * c))**2)
        velocity = self.momentum / (gamma * m_e)
        return velocity, gamma

    def update(self, campo_quadrupolo_k, nivel_ruido):
        # 1. Obter velocidade atual
        v, gamma = self.get_velocity()
        
        # 2. Calcular Campo Magnético (Quadrupolo Idealizado)
        # Quadrupolos focam em um eixo e desfocam no outro, mas 
        # aqui simulo uma "lente ideal" que foca radialmente para simplificar
        # B é zero no centro (0,0) e cresce com a distância
        x, y, z = self.pos
        
        # Campo magnético corretivo (B deve gerar força contra o desvio)
        # F = q(v x B). Assumindo vz dominante.
        # Para forca restauradora em X: Fx ~ -x -> requer By proporcional a x
        Bx = -campo_quadrupolo_k * y 
        By = -campo_quadrupolo_k * x
        Bz = 0
        B = np.array([Bx, By, Bz])
        
        # 3. Força de Lorentz (Correção)
        F_lorentz = q_e * np.cross(v, B)
        
        # 4. Força de Ruído (Aleatória)
        # Adiciona um "chute" aleatório transversal
        ruido = np.random.normal(0, nivel_ruido, 3)
        ruido[2] = 0 # Sem ruído na direção Z (longitudinal) por enquanto
        
        F_total = F_lorentz + ruido
        
        # 5. Integração (Euler)
        # Atualiza Momento (F = dp/dt)
        self.momentum += F_total * dt
        
        # Atualiza Posição (v = dr/dt)
        # Recalcula v com novo momento para garantir precisão
        v_new, _ = self.get_velocity()
        self.pos += v_new * dt
        
        # Salva histórico
        self.traj_x.append(self.pos[0])
        self.traj_y.append(self.pos[1])
        self.traj_z.append(self.pos[2])

# --- RODANDO A SIMULAÇÃO ---

# Inicializa elétron levemente desalinhado (x=1mm, y=-0.5mm)
# Viajando a 99.9% da velocidade da luz
eletron = ElectronBeam(x0=0.001, y0=-0.0005, energia_v=0.999)

# Força da lente magnética e do ruído
for _ in range(steps):
    eletron.update(campo_quadrupolo_k=5.0, nivel_ruido=1e-22)

# --- VISUALIZAÇÃO (Estilo Boids) ---
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))

# Gráfico 1: Vista Superior (Desvio em X ao longo de Z)
ax1.plot(eletron.traj_z, eletron.traj_x, 'b-', label='Trajetória')
ax1.axhline(0, color='k', linestyle='--', alpha=0.3, label='Centro do Tubo')
ax1.set_title("Correção de Trajetória (Vista Superior)")
ax1.set_xlabel("Distância Percorrida Z (m)")
ax1.set_ylabel("Desvio Lateral X (m)")
ax1.legend()
ax1.grid(True)

# Gráfico 2: Plano Transversal (Onde o feixe "bate")
ax2.plot(eletron.traj_x, eletron.traj_y, '.-', alpha=0.5)
ax2.plot(eletron.traj_x[0], eletron.traj_y[0], 'go', label='Início')
ax2.plot(eletron.traj_x[-1], eletron.traj_y[-1], 'ro', label='Fim')
ax2.set_title("Movimento Transversal (XY)")
ax2.set_xlabel("X (m)")
ax2.set_ylabel("Y (m)")
ax2.legend()
ax2.grid(True)
ax2.axis('equal')

plt.tight_layout()
plt.show()